In [ ]:
#this is a roadmap from beginning truth to the final iteration of iPto19

In [2]:
import cobra
import pandas as pd

V1 was originally created from the feeder document, which was derived from looking at reactions present in the draft reconstruction and confirming the activity and presence of an enzyme. This is the document that starts the whole process. It was converted first into an sbml file format to be later read as a cobra model object. 

In [ ]:
#this is a general excel script that is used to convert each of the excel files into a json model, which is then converted into an sbml model for ease of access
#generate the list of reactions to be used for generating the model, from the curation file
xl_file = pd.read_excel('../data/PST_feeder.xlsx')

#at this point we have a grab bag of reactions ready to be inserted into the model, which we have yet to create
universal = cobra.io.load_json_model('../data/modelseed_data/seed_universal.json') #load the grab bag
universal.reactions
for x in universal.reactions:
    x.id = x.id + "0" #this is to match the formating of the rxn id in the PST model
for y in universal.metabolites:
    y.id = y.id + "0" #formatting again to match previous model generated
universal.metabolites
#reaction ids now match those found in pst

PST = Model('PST') #empty model

#add reactions from the grab bag
for i in range(0,len(xl_file)):
    reaction_row = xl_file.iloc[i,]
    reaction_id = reaction_row['Reaction ID']
    reaction_to_add = universal.reactions.get_by_id(reaction_id).copy()
    reaction_to_add.subsystem = reaction_row ['Subsystems'] #add subsystem for the reaction 
    reaction_to_add.gene_reaction_rule = reaction_row['GPR'] #add gpr for the reaction
    PST.add_reaction(reaction_to_add)

#write the model
cobra.io.save_json_model(PST,"../results/reconstructions/pst_feeder.json")
pst_feeder = cobra.io.load_json_model('../results/reconstructions/pst_feeder.json')
cobra.io.write_sbml_model(pst_feeder, filename='../results/reconstructions/pst_feeder.sbml', use_fbc_package = True)


In [7]:
pst_feeder = cobra.io.read_sbml_model("../results/reconstructions/pst_feeder.sbml")
pst_feeder

Name,PST
Memory address,0x07ff740783f28
Number of metabolites,242
Number of reactions,149
Number of groups,0
Objective expression,0
Compartments,"c, e"


V1 was not integrated with the orthologs between PAO1 and Pst yet. So, there are a few scripts and outputs that achive this. Originally, this was done through the scripts "ortholog_compiler", "comp_list", "rxns_left", "rxn_list_comp" all feeding into the newly updated PST_curation.xlsx. I will combine all the scripts here in the next cells.

In [ ]:
#generate the list of reactions to be used for generating the model, from the curation file
#this is from the original script "xl_to_model_pao1.py"

xl_file = pd.read_excel('../data/SupplementaryData4_mPAO1.xlsx')

#at this point we have a grab bag of reactions ready to be inserted into the model, which we have yet to create
universal = cobra.io.load_json_model('../data/seed_universal.json') #load the grab bag
universal.reactions
for x in universal.reactions:
    x.id = x.id + "0" #this is to match the formating of the rxn id in the PST model
for y in universal.metabolites:
    y.id = y.id + "0" #formatting again to match previous model generated
universal.metabolites
#reaction ids now match those found in pst

PA = Model('PA') #empty model
pa_nan=[]
#add reactions from the grab bag
for i in range(0,len(xl_file)):
    reaction_row = xl_file.iloc[i,]
    reaction_id = reaction_row['Abbreviation'] + '_c0'
 #   print (reaction_id)
    reaction_to_add = universal.reactions.get_by_id(reaction_id).copy()
    reaction_to_add.subsystem = reaction_row ['Subsystems'] #add subsystem for the reaction 
 #   pst_nan.append(reaction_to_add.subsystem)   
    reaction_to_add.gene_reaction_rule = reaction_row['GPR'] #add gpr for the reaction
    PA.add_reaction(reaction_to_add)
    print (PA.reactions.get_by_id(reaction_id))
#print (pst_nan)
#print ([g.id for g in PST.genes])
#print ([m.id for m in PST.metabolites])
gen = [rxn.gene_reaction_rule for rxn in PA.reactions]
#print (PST.reactions[0].gene_reaction_rule)
#write the model
#for x in gen:
 #   if type(x) is not str:
  #      print(x)
#print (gen)
cobra.io.save_json_model(PA,"pao1.json")
#do the reactions have gprs and do the contain wht you expect




In [ ]:
#this is from the original script "comp_list.py" which was used to create a list of reactions not in the current curation document and compare it to the draft model to see
#which reactions were included in each. This will generate a list of reactions for further curation called PST_updates_list.
#import the excel file for PST_curation
import pandas as pd
xl_file = pd.read_excel('../data/PST_curation.xlsx')

#import the reaction list from the draft
rxn_list = pd.read_csv('../PST_reaction_draft.csv')

#create new list blank
not_in_curation = []
just_ids = []
draft_rxns = []
#compare the dataframes

for x in range(0, len(xl_file)):
    reaction_row = xl_file.iloc[x,]
    n_id = reaction_row['Reaction ID']
    just_ids.append(n_id)
print (just_ids)

for x in range(0, len(rxn_list)):
    reaction_row_m = rxn_list.iloc[x,]
    m_id = reaction_row_m['id']
    draft_rxns.append(m_id)
print (draft_rxns)

for x in draft_rxns:
    if x not in just_ids:
        not_in_curation.append(x)
    else:
        pass
print (not_in_curation)

#df = pd.DataFrame(not_in_curation)
#print (df)
#df.to_csv("/Users/pjt7wd/Desktop/PST_updates_list.csv")

In [24]:
#this is the script originally called "ortholog_to_list.py" which was used to compare the PAO1 and PST orthologs to the list of PAO1 gprs in PAO1 reactions
#followed by comparing the list of PAO1 reactions to the list of reactions left to be curated in PST draft. If there is an ortholog between PAO1 and PSt, and it has a
#matching reaction that is defined in PAO1, the reaction will be added to a list with updated orthologous genes in PST.
#read the two library files for the lists, as well as the reaction list to call reactions
supp_pao1 = pd.read_excel("../data/previous_data_from_curation/SupplementaryData4_mPAO1.xlsx")
orthologs = pd.read_csv('../data/previous_data_from_curation/PA01_PSY_orthologs.csv')
rxn_list = pd.read_csv('../data/previous_data_from_curation/PST_updates_list.csv')
print (rxn_list)

#create reaction list to call in the supp
rxns = []
for x in range(0,len(rxn_list)):
    reaction_row = rxn_list.iloc[x,]
    ids = reaction_row['0']
    rxns.append(ids)
print (rxns)

#now we have the reactions extracted into a list to compare with the pa01 library, which must be similarly compared
#when comparing the reaction, create a dictionary with the reaction id as the key and the genes as the value
pao1_genes = {}

#extract the reactions from pao1 and the corresponding genes to create a dictionary
for x in range(0,len(supp_pao1)):
    reaction_row = supp_pao1.iloc[x,]
    if "rxn" in reaction_row["Abbreviation"]:
        rxn_abb = reaction_row['Abbreviation'] + "_c0"
        rxn_genes = reaction_row['Genes']
    print (rxn_genes)
    print (rxn_abb)
    #populate the dictionary with the keys (rxn abbreviations) and values (gene ids)
    pao1_genes[rxn_abb] = rxn_genes
    print (pao1_genes)    

#now compare the dictionary keys (rxns) to the list of rxns in the PST update list of rxns, rxn_list
matching_rxns = {}
for k in rxns:
    if k in pao1_genes:
        print (k, pao1_genes[k])
        str_pao1_genes = str(pao1_genes[k])
        split_genes = str.split(str_pao1_genes, ' ')
        print (split_genes)
        matching_rxns[k] = split_genes        
        print (matching_rxns)

#this has the format that I want, but now add to a pandas dataframe
df_matching_rxns = pd.DataFrame.from_dict(matching_rxns, orient='index')
print (df_matching_rxns)

#dataframe is now in the format to be used. it includes the reaction ids and the associated genes
#now I must match the asociated genes with the orthologous pairs
#first create dictionary with the orthologous pairs
ortholog_dict = {}
print (orthologs)
for x in range(0, len(orthologs)):
    pair_row = orthologs.iloc[x,]
    query = pair_row['Locus Tag (Query)']
    result = pair_row['Locus Tag (Hit)']
    ortholog_dict[query]=result
    print (ortholog_dict)

#the ortholog dict is assembled, with each query value (all Pao1 genes and PSPTO genes were surveyed)
#this dict will now be iterated over to match with the missing reactions PAo1 genes
rxn_PST = {}
for k,v in matching_rxns.items():
    for x in matching_rxns[v]:
        if x in ortholog_dict:
            print (k ,v, ortholog_dict[x])
            rxn_PST[k] = ortholog_dict[x]
            print (rxn_PST)
        else:
            pass
df_match_orthologs = pd.DataFrame.from_dict(rxn_PST, orient='index') 
print(df_match_orthologs)   
df_match_orthologs.to_csv("../data/PST_gpr.csv")
#the above code will only produce matches for all singly annotated reactions with one gene annoted
#plural genes have not been matched this way
for k in matching_rxns.items():
    if k in ortholog_dict:
        print (k, v, ortholog_dict[k])
    else:
        print (k ,v, 'no')
        pass
     #   df_matching_rxns[]
        
#the resulting csv file was combined with the original PST curation file and loaded into an sbml model
#once this is combined
import pandas as pd
import cobra
from cobra import Model, Metabolite, Gene, Reaction

NameError: name 'pd' is not defined

In [9]:
#this is generated from the curation excel file, which has genes and recaction annotations, but the biomass equation was added from iPAE48, (the PAO1 model from the Papin lab UVA)
#this is a general excel script that is used to convert each of the excel files into a json model, which is then converted into an sbml model for ease of access
#generate the list of reactions to be used for generating the model, from the curation file
xl_file = pd.read_excel('../data/PST_curation.xlsx')

#at this point we have a grab bag of reactions ready to be inserted into the model, which we have yet to create
universal = cobra.io.load_json_model('../data/modelseed_data/seed_universal.json') #load the grab bag
universal.reactions
for x in universal.reactions:
    x.id = x.id + "0" #this is to match the formating of the rxn id in the PST model
for y in universal.metabolites:
    y.id = y.id + "0" #formatting again to match previous model generated
universal.metabolites
#reaction ids now match those found in pst

PST = Model('PST') #empty model

#add reactions from the grab bag
for i in range(0,len(xl_file)):
    reaction_row = xl_file.iloc[i,]
    if "PSPTO" in reaction_row['GPR']:
        reaction_id = reaction_row['Reaction ID']
        reaction_to_add = universal.reactions.get_by_id(reaction_id).copy()
        reaction_to_add.subsystem = reaction_row ['Subsystems'] #add subsystem for the reaction 
        reaction_to_add.gene_reaction_rule = reaction_row['GPR'] #add gpr for the reaction
        PST.add_reaction(reaction_to_add)

#write the model
cobra.io.save_json_model(PST,"../results/reconstructions/pst_2.json")
pst_2 = cobra.io.load_json_model('../results/reconstructions/pst_2.json')
cobra.io.write_sbml_model(pst_2, filename='../results/reconstructions/pst_2.sbml', use_fbc_package = True)
pst_2 = cobra.io.read_sbml_model('../results/reconstructions/pst_2.sbml')
#the inclusion of a biomass equation from PAO1
pa = cobra.io.read_sbml_model('../data/previous_reconstructions/iPAE1146.xml')
bio1 = pa.reactions.PAO1_biomass.copy()
bio2 = pa.reactions.PAO1_biomass_v13ub.copy()
bio3 = pa.reactions.PAO1_biomass_v13.copy()
bio4 = pa.reactions.PAO1_biomass_v4.copy()
pst_2.add_reactions([bio1, bio2, bio3, bio4])
pst_2 = cobra.io.read_sbml_model("../results/reconstructions/PST_pao1_temp_biomass_v2.sbml")

Name,PST
Memory address,0x07ff750cdd128
Number of metabolites,1025
Number of reactions,752
Number of groups,0
Objective expression,1.0*PAO1_Biomass - 1.0*PAO1_Biomass_reverse_55fa3
Compartments,"c, e"


In [45]:
pst_2 = cobra.io.read_sbml_model("../results/reconstructions/PST_pao1_temp_biomass_v2.sbml")
pst_2.exchanges
for rxn in pst_2.exchanges:
    rxn.id = str(rxn.id[:-17]) + '_e'
    print (rxn.id)
cobra.io.write_sbml_model(pst_2, "../results/reconstructions/PST_pao1_temp_biomass_v3.sbml")

EX_cpd00797_e
EX_cpd17085_e
EX_cpd00176_e
EX_cpd00024_e
EX_cpd00489_e
EX_cpd00029_e
EX_cpd00142_e
EX_cpd00182_e
EX_cpd11587_e
EX_cpd11584_e
EX_cpd11593_e
EX_cpd11583_e
EX_cpd11586_e
EX_cpd11582_e
EX_cpd01092_e
EX_cpd00162_e
EX_cpd01262_e
EX_cpd01048_e
EX_cpd04098_e
EX_cpd00540_e
EX_cpd11596_e
EX_cpd00211_e
EX_cpd00063_e
EX_cpd00635_e
EX_cpd01012_e
EX_cpd00098_e
EX_cpd11595_e
EX_cpd00137_e
EX_cpd00099_e
EX_cpd00011_e
EX_cpd00149_e
EX_cpd00058_e
EX_cpd01015_e
EX_cpd01017_e
EX_cpd00381_e
EX_cpd00367_e
EX_cpd00307_e
EX_cpd00117_e
EX_cpd00438_e
EX_cpd00654_e
EX_cpd00277_e
EX_cpd03279_e
EX_cpd00412_e
EX_cpd00082_e
EX_cpd00072_e
EX_cpd00653_e
EX_cpd00280_e
EX_cpd00609_e
EX_cpd00027_e
EX_cpd00079_e
EX_cpd00221_e
EX_cpd01307_e
EX_cpd00386_e
EX_cpd00637_e
EX_cpd00652_e
EX_cpd00105_e
EX_cpd00550_e
EX_cpd11579_e
EX_cpd00021_e
EX_cpd10516_e
EX_cpd03724_e
EX_cpd00106_e
EX_cpd00108_e
EX_cpd00164_e
EX_cpd11581_e
EX_cpd11589_e
EX_cpd00100_e
EX_cpd00080_e
EX_cpd00033_e
EX_cpd00139_e
EX_cpd15603_e
EX_cpd

The difference between PSTv2 and PSTv3 is the addition of exchange reactions with a more representative name. The previous version included "L_PAREN" etc, so all of these were changed and added as EX_metabolite_id. In PSTv3, the former exchange reactions still exsist within the model.

In [6]:
pst_3 = cobra.io.read_sbml_model("../results/reconstructions/PST_pao1_temp_biomass_v3.sbml")
pst_3

Name,PST
Memory address,0x07fdec1093d30
Number of metabolites,1028
Number of reactions,920
Number of groups,0
Objective expression,1.0*PAO1_Biomass - 1.0*PAO1_Biomass_reverse_55fa3
Compartments,"c, e, c0"


Between v3 and v4, there is a change in the compartments included in the model. In v3, compartments c, e, and c0 exsist. This creates inconsitancies that are incompatable for cross compartment movement. c and c0 were collapsed into one compartment, c. There is no change in the number of genes or reactions, but there is one less metabolite in v4. 

In [10]:
pst_4 = cobra.io.read_sbml_model("../results/reconstructions/PST_pao1_temp_biomass_v4.sbml")
len(pst_4.exchanges)

334

To get from v4 to v4 with all annotations is the next step. This is the point where we added a lot of nomenclature for satisfying MEMOTE, including the inchie strings and other database knowledge.

After PSTv4 with all annotations added, first round of ensemble gapfilling took place, where we created an ensemble of 500 members, gapfilled to the biolog growth data, and looked for the most disperate reaction additions in alternative gapfilling solutions for curation.
After the first round was complete, we ended up with model_post_ensemble_curation_round1.xml. This was followed with a second wave of ensemble curation, with changes made to the base model to fill in the curated reactions, and another 500 member ensebmle was created. Again the most disperate reactions were found within the solutions, and these were targeted for curation.
The framework of execution can be in the file "gapfill_to_ensemble.ipynb", which takes any clear positive growth on biolog data (>0.3OD) and creates an exchange reaction for the substrate. Gapfilling solutions are generated based on this script, where the number of ensemble members can be changed to generate ensembles of different sizes. The framework presented in the example script was used for each ensemble curation step. The next script, ensemble_learning.ipynb is to generate the list of the most disperate reactions, and finally, the output is an excel file with all of the reactions in their disperate form. From this, the script curate_from_ensemble***.ipynb is used as a journal for reaction additions or deletions to the model from the base model of the ensemble.
There is a drastic reduction in the number of metabolites and the number of reactions due to cleaning up exchange reactions and the previous biomass reactions as well.
The exchange reactions were introduced from the information in the Ma and Mhadmi papers, respectivly https://doi.org/10.1073/pnas.1319485111 and https://doi.org/10.1016/j.envexpbot.2014.07.002

In [15]:
pst_5 = cobra.io.read_sbml_model("../results/reconstructions/v5_with_all_annotations.xml")
pst_5

Name,PST
Memory address,0x07fdf020f6e10
Number of metabolites,797
Number of reactions,719
Number of groups,0
Objective expression,0
Compartments,"c, e"


From PSTv5 to PSTv6, the main changes include integrating exchange reactions from the Anderson 2014 paper, which looked at the differences in apoplasmic content of arabidopsis in normal and mkp1 mutants. This information is later used as in planta simulations for ensemble gene deletions. This is done in the notebook new_exchange_rxns2.ipynb.

In [27]:
pst_6 = cobra.io.read_sbml_model("../results/reconstructions/v6_with_all_annotations.xml")
pst_6.exchanges

[<Reaction EX_cpd11416_c at 0x7fdf02c51fd0>,
 <Reaction EX_cpd00794_e at 0x7fdf02c5a9e8>,
 <Reaction EX_cpd00023_e at 0x7fdf02c5afd0>,
 <Reaction EX_cpd00034_e at 0x7fdf02a6e668>,
 <Reaction EX_cpd00041_e at 0x7fdf02c5aac8>,
 <Reaction EX_cpd00048_e at 0x7fdf02c5aa20>,
 <Reaction EX_cpd00053_e at 0x7fdf02c697b8>,
 <Reaction EX_cpd00080_e at 0x7fdf02c69860>,
 <Reaction EX_cpd00107_e at 0x7fdf02c69908>,
 <Reaction EX_cpd00118_e at 0x7fdf02c699b0>,
 <Reaction EX_cpd00119_e at 0x7fdf02c69a58>,
 <Reaction EX_cpd00154_e at 0x7fdf02c69b00>,
 <Reaction EX_cpd00156_e at 0x7fdf02c69ba8>,
 <Reaction EX_cpd00210_e at 0x7fdf02c69c50>,
 <Reaction EX_cpd00264_e at 0x7fdf02c69cf8>,
 <Reaction EX_cpd00322_e at 0x7fdf02c69da0>,
 <Reaction EX_cpd00637_e at 0x7fdf02c69e48>,
 <Reaction EX_cpd00205_e at 0x7fdf02c69ef0>,
 <Reaction EX_cpd00971_e at 0x7fdf02c69f98>,
 <Reaction EX_cpd00067_e at 0x7fdf02c69fd0>,
 <Reaction EX_cpd00060_e at 0x7fdf02c78128>,
 <Reaction EX_cpd00129_e at 0x7fdf02c781d0>,
 <Reaction

PSTv6 integrated the exchange reactions in an improper form, and this rendered them to not be marked as exchange reactions. To fix this, I added _e to the metabolite identifiers at the end of the exchange reactions in the original script of new_exchange_rxns2.ipynb. Further additions to PSTv6 include the LPS synthesis pathway. The two notebooks necessary for these changes are new_exchange_rxns2.ipynb and lps_ciration.ipynb.

In [21]:
pst_6 = cobra.io.read_sbml_model("../results/reconstructions/psy6_ex_fixed.xml")
pst_6.reactions

[<Reaction rxn00001_c at 0x7f8142e29400>,
 <Reaction rxn00002_c at 0x7f8142e293c8>,
 <Reaction rxn00011_c at 0x7f8142e2b898>,
 <Reaction rxn00012_c at 0x7f8142e2b828>,
 <Reaction rxn00020_c at 0x7f8182270ba8>,
 <Reaction rxn00025_c at 0x7f8142e2b860>,
 <Reaction rxn00029_c at 0x7f8142e2f7b8>,
 <Reaction rxn00048_c at 0x7f8142e2bcc0>,
 <Reaction rxn00055_c at 0x7f8142e2fa90>,
 <Reaction rxn00056_c at 0x7f8142e2fcf8>,
 <Reaction rxn00060_c at 0x7f8142e2fd30>,
 <Reaction rxn00062_c at 0x7f8142e32cf8>,
 <Reaction rxn00065_c at 0x7f8142e2fa58>,
 <Reaction rxn00077_c at 0x7f8142e34d68>,
 <Reaction rxn00083_c at 0x7f8142e36e80>,
 <Reaction rxn00085_c at 0x7f8142e36ef0>,
 <Reaction rxn00086_c at 0x7f8142e38e48>,
 <Reaction rxn00095_c at 0x7f81822b94a8>,
 <Reaction rxn00097_c at 0x7f8142e2bc50>,
 <Reaction rxn00100_c at 0x7f8142e3dfd0>,
 <Reaction rxn00101_c at 0x7f8142e3ffd0>,
 <Reaction rxn00105_c at 0x7f8142e3ff98>,
 <Reaction rxn00113_c at 0x7f8142e428d0>,
 <Reaction rxn00117_c at 0x7f8142e

The next major addition was the coronatine synthesis pathway, which was performed in cor_addition.ipynb. This went two ways: the first attempt was to add each of the reactions independantly that were laid out in https://doi.org/10.1073/pnas.95.26.15469 . This is a putative pathway, but the most worked out one to date. However due to issues of mass balance in ACP, I collapsed the pathway to more reflect the basic syntehsis posed in Pseudomonas syringae Phytotoxins: Mode of Action, Regulation, and Biosynthesis by Peptide and Polyketide Synthetases (no doi presented).

In [30]:
pst_6 = cobra.io.read_sbml_model("../results/reconstructions/pst6_cor.xml")
pst_6.reactions

[<Reaction rxn00001_c at 0x7f8133077400>,
 <Reaction rxn00002_c at 0x7f81330773c8>,
 <Reaction rxn00011_c at 0x7f813307b860>,
 <Reaction rxn00012_c at 0x7f813307b7f0>,
 <Reaction rxn00020_c at 0x7f8132e5b748>,
 <Reaction rxn00025_c at 0x7f813307b828>,
 <Reaction rxn00029_c at 0x7f813307f780>,
 <Reaction rxn00048_c at 0x7f813307bc88>,
 <Reaction rxn00055_c at 0x7f813307fa58>,
 <Reaction rxn00056_c at 0x7f813307fcc0>,
 <Reaction rxn00060_c at 0x7f813307fcf8>,
 <Reaction rxn00062_c at 0x7f8133081cc0>,
 <Reaction rxn00065_c at 0x7f813307fa20>,
 <Reaction rxn00077_c at 0x7f8133084d30>,
 <Reaction rxn00083_c at 0x7f81515c14e0>,
 <Reaction rxn00085_c at 0x7f81515c10f0>,
 <Reaction rxn00086_c at 0x7f813308ae10>,
 <Reaction rxn00095_c at 0x7f813308c908>,
 <Reaction rxn00097_c at 0x7f813307bc18>,
 <Reaction rxn00100_c at 0x7f8151600fd0>,
 <Reaction rxn00101_c at 0x7f8151602f98>,
 <Reaction rxn00105_c at 0x7f8151602fd0>,
 <Reaction rxn00113_c at 0x7f8151608898>,
 <Reaction rxn00117_c at 0x7f81516

To PSTv6, the next major change was the addition of the PST specific biomass equation, generating PSTv7. This is done through the notebook concise_gapfill.ipynb, which is a part of the gapfilling protocol. So, even though it is embedded in the gapfilling protocol, this will not change teh outcome bcause it is drawing from the same source everytime. There is no additional model based with the actual biomass equation, however, using the adapted version of the gapfilling protocol, I have generated PSTv7 with the new biomass equation.

In [37]:
pst_7 = cobra.io.read_sbml_model("../results/reconstructions/pstv7.xml")
pst_7

Name,PST
Memory address,0x07f81431e6e48
Number of metabolites,840
Number of reactions,756
Number of groups,0
Objective expression,0
Compartments,"c, e"


PSTv7 underwent another gapfilling protocol to have one more final curation step, in which all the added reactions were queried for potential additional gprs to add to the given reactions. This was done in the notebook gpr_filler.ipynb, aptly named.
The resulting base model is named PSTv8.

In [47]:
pst_8 = cobra.io.read_sbml_model("../results/reconstructions/pstv8.xml")
pst_8

Name,PST
Memory address,0x07f817179efd0
Number of metabolites,1020
Number of reactions,1208
Number of groups,0
Objective expression,0
Compartments,"c, e"


PSTv8 can now undergo a final gapfilling step using the ensemble method to simulated growth on media and other analysis presented in this paper.